In [1]:
import os
import numpy as np
from tensorflow.keras.applications import DenseNet121


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data_dir = '/Users/saikarthik/Downloads/untitled folder/parkinsons_dataset'


In [3]:
img_height, img_width = 224, 224 
batch_size = 32


In [4]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,  
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)


In [5]:

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
)

Found 665 images belonging to 2 classes.


In [6]:

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
)

Found 166 images belonging to 2 classes.


In [7]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

2025-04-02 16:00:27.938285: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-02 16:00:27.938323: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-02 16:00:27.938337: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-02 16:00:27.938411: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-02 16:00:27.938603: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


29084464/29084464 [==============================] - 9s 0us/step


In [8]:
for layer in base_model.layers:
    layer.trainable = False

In [9]:


model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


In [10]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
)

Epoch 1/100


2025-04-02 16:00:42.498916: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - ETA: 0s - loss: 0.7941 - accuracy: 0.6421

2025-04-02 16:00:50.038850: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


21/21 [==============================] - 13s 418ms/step - loss: 0.7941 - accuracy: 0.6421 - val_loss: 0.5281 - val_accuracy: 0.7470
Epoch 2/100
21/21 [==============================] - 6s 250ms/step - loss: 0.6916 - accuracy: 0.6767 - val_loss: 0.4720 - val_accuracy: 0.8012
Epoch 3/100
21/21 [==============================] - 5s 245ms/step - loss: 0.5479 - accuracy: 0.7414 - val_loss: 0.4369 - val_accuracy: 0.7952
Epoch 4/100
21/21 [==============================] - 5s 256ms/step - loss: 0.4845 - accuracy: 0.7805 - val_loss: 0.4649 - val_accuracy: 0.7651
Epoch 5/100
21/21 [==============================] - 5s 246ms/step - loss: 0.4328 - accuracy: 0.8135 - val_loss: 0.4147 - val_accuracy: 0.7651
Epoch 6/100
21/21 [==============================] - 5s 248ms/step - loss: 0.4063 - accuracy: 0.8256 - val_loss: 0.3580 - val_accuracy: 0.8133
Epoch 7/100
21/21 [==============================] - 5s 245ms/step - loss: 0.3638 - accuracy: 0.8436 - val_loss: 0.3626 - val_accuracy: 0.8133
Epoch 8/10

In [15]:
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

validation_generator.reset()
predictions = (model.predict(validation_generator) > 0.5).astype("int64")
true_classes = validation_generator.classes

print("\nClassification Report:")
print(classification_report(true_classes, predictions))

print("\nConfusion Matrix:")
print(confusion_matrix(true_classes, predictions))

6/6 [==============================] - 1s 154ms/step - loss: 0.0869 - accuracy: 0.9699
Validation Accuracy: 96.99%
6/6 [==============================] - 1s 174ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.75      0.73       122
           1       0.17      0.14      0.15        44

    accuracy                           0.59       166
   macro avg       0.44      0.45      0.44       166
weighted avg       0.56      0.59      0.58       166


Confusion Matrix:
[[92 30]
 [38  6]]
